In [ ]:
from copydetect import CopyDetector
import json
import os
import shutil

## --- PARAMETER ---
CONTEST = "W405" # Contest name
QUESTION = '/Q3'
LANG = "cpp" # the file extension gonna deal with

In [ ]:
# create folder
try:
    os.mkdir(CONTEST+"/test")
except:
    pass

cheater = []
for i in range(12):
    detector = CopyDetector(test_dirs=[CONTEST+QUESTION+"/"+str(i)+"000-"+str(i)+"999", CONTEST+"/test"], extensions=[LANG], display_t=0.95)
    detector.run()
    copied_code = detector.get_copied_code_list()
    print(len(copied_code))
    
    for row in copied_code:
        p1 = row[0]
        p2 = row[1]
        user1 = int(row[2][row[2].rindex("\\")+1:-len(LANG)-1])
        user2 = int(row[3][row[3].rindex("\\")+1:-len(LANG)-1])
    
        group_1 = -1;
        group_2 = -2;
        
        for j, group in enumerate(cheater):
            if user1 in group:
                group_1 = j
            if user2 in group:
                group_2 = j
    
        if p1 >= 1 and p2 >= 1:
            if group_1 == -1 and group_2 == -2:
                cheater.append([user1, user2])
            elif group_1 == -1:
                cheater[group_2].append(user1)
            elif group_2 == -2:
                cheater[group_1].append(user2)
            elif group_1 != group_2:
                cheater[group_1].extend(cheater[group_2])
                del cheater[group_2]

    for group in cheater:
        folder = '/'+str(group[0]//1000)+"000-"+str(group[0]//1000)+"999/"
    
        shutil.copyfile(CONTEST+QUESTION+folder+str(group[0])+'.'+LANG, CONTEST+"/test/"+str(group[0])+'.'+LANG)

print("[")
for i in cheater:
    i.sort()
    print(i, ",")
print("]")

for group in cheater:
    for idx in range(1, len(group)):
        folder = '/'+str(group[idx]//1000)+"000-"+str(group[idx]//1000)+"999/"
        
        if os.path.exists(CONTEST+QUESTION+folder+str(group[idx])+'.'+LANG):
            os.remove(CONTEST+QUESTION+folder+str(group[idx])+'.'+LANG)

JSON = CONTEST+QUESTION+"_"+LANG+".json"

with open(JSON, "w") as f:
    json.dump(cheater, f)